<a href="https://colab.research.google.com/github/Raulfb04/Administraci-n-de-la-producci-n/blob/main/programacion%20lineal%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.optimize import linprog

# =========================================================
# EJERCICIO 1 - Programación Lineal
# Variables:
# x1 = unidades a fabricar del producto p1
# x2 = unidades a fabricar del producto p2
#
# Max Z = 10*x1 + 5*x2
#
# Restricciones:
# (1) x1 + x2 <= 27500               (capacidad)
# (2) 15*x1 + 20*x2 <= 500000        (presupuesto)
# (3) x1 = 3*x2                      (relación de demanda)
# (4) x1 + x2 >= 70000   (si se interpreta literal; pero choca con (1))
# x1, x2 >= 0
# =========================================================

# linprog MINIMIZA, por eso ponemos la función objetivo con signo negativo
c = [-10, -5]

# Restricciones tipo <= (A_ub x <= b_ub)
A_ub = [
    [1, 1],        # capacidad
    [15, 20]       # costo total
]
b_ub = [27500, 500000]

# Restricción de igualdad (A_eq x = b_eq): x1 - 3x2 = 0
A_eq = [[1, -3]]
b_eq = [0]

# No negatividad
bounds = [(0, None), (0, None)]

# -------------------------
# CASO A: incluir x1 + x2 >= 70000 (lo pasamos a <= multiplicando por -1)
# -(x1 + x2) <= -70000
# -------------------------
A_ub_A = A_ub + [[-1, -1]]
b_ub_A = b_ub + [-70000]

res_A = linprog(c, A_ub=A_ub_A, b_ub=b_ub_A, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

print("=== CASO A (con demanda total >= 70000) ===")
print("Status:", res_A.message)
if res_A.success:
    x1_A, x2_A = res_A.x
    print(f"x1 = {x1_A:.2f}, x2 = {x2_A:.2f}")
    print("Z max =", -res_A.fun)
else:
    print("No hay solución factible (las restricciones se contradicen).")

# -------------------------
# CASO B: sin la restricción >= 70000 (demanda solo como 'hay mercado suficiente')
# -------------------------
res_B = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

print("\n=== CASO B (sin demanda total >= 70000) ===")
print("Status:", res_B.message)
if res_B.success:
    x1_B, x2_B = res_B.x
    print(f"x1 = {x1_B:.2f}, x2 = {x2_B:.2f}")
    print("Z max =", -res_B.fun)

    # Holguras (slacks) de restricciones <=
    # slack = b - A*x  (si slack=0, esa restricción es activa)
    print("\nHolguras:")
    slack_capacidad = 27500 - (x1_B + x2_B)
    slack_costo = 500000 - (15*x1_B + 20*x2_B)
    print("Capacidad (slack) =", slack_capacidad)
    print("Costo (slack)     =", slack_costo)
else:
    print("Algo salió mal, revisa el planteamiento.")

=== CASO A (con demanda total >= 70000) ===
Status: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is None)
No hay solución factible (las restricciones se contradicen).

=== CASO B (sin demanda total >= 70000) ===
Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
x1 = 20625.00, x2 = 6875.00
Z max = 240625.0

Holguras:
Capacidad (slack) = 0.0
Costo (slack)     = 53125.0


## Conclusión – Ejercicio 1

Al incluir la restricción de demanda total $x_1 + x_2 \ge 70000$,
el modelo se vuelve no factible debido a que contradice la restricción
de capacidad $x_1 + x_2 \le 27500$. Por lo tanto, no existe una solución
que satisfaga simultáneamente ambas condiciones.

Interpretando la demanda total mayor a 70,000 unidades como una condición
de mercado (es decir, que existe demanda suficiente para vender todo lo producido)
y no como una restricción mínima obligatoria, el modelo resulta factible.

La solución óptima encontrada mediante el solver es:

$x_1 = 20625, \quad x_2 = 6875$

con una utilidad máxima de:

$Z_{\max} = 240625$

La restricción de capacidad es activa (holgura = 0), por lo que se utiliza
el 100% de la capacidad instalada. En cambio, la restricción presupuestal
presenta una holgura de \$53,125, indicando que el presupuesto no se agota
completamente.